<a href="https://colab.research.google.com/github/xprilion/gemini-as-a-judge-for-rag-evals/blob/main/Step_1_Problem_Context_The_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini As A Judge for RAG Evals

## The RAG

### 1. Load the datasets

In [10]:
!wget https://raw.githubusercontent.com/xprilion/gemini-as-a-judge-for-rag-evals/refs/heads/main/reviews.json

--2025-03-01 08:04:16--  https://raw.githubusercontent.com/xprilion/gemini-as-a-judge-for-rag-evals/refs/heads/main/reviews.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 507865 (496K) [text/plain]
Saving to: ‘reviews.json’

reviews.json        100%[===================>] 495.96K  --.-KB/s    in 0.05s   

2025-03-01 08:04:16 (9.94 MB/s) - ‘reviews.json’ saved [507865/507865]



### Packages

In [11]:
%%capture
!pip install qdrant-client[fastembed]
!pip install google-genai

### Imports

In [12]:
import pandas as pd
import json
import os
import time
from tqdm import tqdm
from google import genai
from google.genai import types
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import uuid

from google.colab import userdata

### Helpers

In [13]:
collection_name = "product_reviews"

In [14]:
GEMINI_KEY = userdata.get('GEMINI_API_KEY')
gemini_client = genai.Client(
    api_key=GEMINI_KEY
)

In [15]:
def getGeminiResponse(prompt, max_tokens=8192, response_type="text/plain"):
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text=prompt
                ),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        top_p=0.95,
        top_k=40,
        max_output_tokens=max_tokens,
        response_mime_type=response_type,
    )
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", contents=contents, config=generate_content_config
    )
    return response.text

In [16]:
getGeminiResponse("What is 2+3?")

'2 + 3 = 5\n'

### EDA

In [17]:
df = pd.read_json('reviews.json')

In [18]:
df.head()

,product_title,product_description,review
0,Hammer,This 16 oz claw hammer is perfect for general ...,I've been using this hammer for a few months n...
1,Hammer,This 16 oz claw hammer is perfect for general ...,This hammer is a solid addition to my toolbox....
2,Hammer,This 16 oz claw hammer is perfect for general ...,I purchased this hammer for some home renovati...
3,Hammer,This 16 oz claw hammer is perfect for general ...,"As a professional carpenter, I rely on my tool..."
4,Hammer,This 16 oz claw hammer is perfect for general ...,This hammer is a great value for the price. Th...


### Connect Qdrant

In [19]:
QDRANT_URL = "https://qdrant-1.sg-1.cloudtop.dev"
QDRANT_KEY = userdata.get('PERSONAL_QDRANT_KEY')

In [20]:
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_KEY, port=None)

### Create Documents

In [21]:
documents = []
metadatas = []
ids = []

In [22]:
for index, row in df.iterrows():
    product_title = row['product_title']
    product_description = row['product_description']
    review = row['review']

    # Combine product information and review into a single document
    document = f"Title: {product_title}\nDescription: {product_description}\nReview: {review}"

    # Create metadata dictionary
    metadata = {
        "product_title": product_title,
        "product_description": product_description,
        "review": review,
        "index": index, # add index for reference
    }

    # Generate a unique ID
    doc_id = str(uuid.uuid4())

    documents.append(document)
    metadatas.append(metadata)
    ids.append(doc_id)

In [23]:
len(documents)

900

### Load data into qdrant

In [24]:
%%capture

qdrant_client.add(
    collection_name=collection_name,
    documents=documents,
    metadata=metadatas,
    ids=ids
)

### Test Retrieval

In [25]:
user_query = "What are the key features of the heavy-duty workbench?"

In [26]:
search_result = qdrant_client.query(collection_name=collection_name, query_text=user_query)

In [27]:
for result in search_result:
    print(result.document)
    print("---")

Title: Workbench
Description: A heavy-duty workbench with a large work surface and integrated power strip. The pegboard back allows for convenient tool storage.
Review: I've had this workbench for a few months now, and it has been a great addition to my workshop. The large work surface is perfect for my projects, and the integrated power strip is very convenient. The pegboard back is a nice feature for keeping my tools organized and within reach. The bench is very sturdy and can handle heavy loads without any issues. Assembly was straightforward, and the instructions were easy to follow. Highly recommend this workbench.
---
Title: Workbench
Description: A heavy-duty workbench with a large work surface and integrated power strip. The pegboard back allows for convenient tool storage.
Review: I bought this workbench for my garage, and it has been a great addition. The large work surface is perfect for my projects, and the integrated power strip is very convenient. The pegboard back is a n

In [28]:
def getRagResponse(question):
    search_result = qdrant_client.query(collection_name=collection_name, query_text=question)
    system_prompt = """
      You are an intelligent assistant designed to provide accurate and informative answers based on retrieved documents.

      Your primary task is to:

      Understand the user's query.
      Retrieve relevant information from the provided context (documents).
      Synthesize the retrieved information into a coherent and accurate response.

      documents:

      """

    documents_text = ""

    doc_count = 1
    for result in search_result:
      documents_text += str(doc_count) + ": \n" + result.document + "\n\n"
      doc_count += 1

    users_query = "\n\n The user is asking: " + question

    prompt = system_prompt + documents_text + users_query

    response = getGeminiResponse(prompt)

    return response

### Test RAG

In [29]:
getRagResponse(user_query)

'Based on the provided documents, the key features of the heavy-duty workbench are:\n\n*   **Large work surface:** Provides ample space for various projects.\n*   **Integrated power strip:** Offers convenient access to power for tools.\n*   **Pegboard back:** Allows for organized tool storage and easy access.\n*   **Heavy-duty construction:** Ensures sturdiness and stability, even with heavy loads.'